In [1]:
# from rest_framework.response import Response
# from rest_framework.views import APIView
# from edge.stats.algorithms.filter1 import  filter
# from edge.stats.spark_utils import read_file
import json
# from edge.stats.models import Filter
# from edge.stats.db_update.col_map import colmap_swap,colmap_json
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from boruta import BorutaPy
from sklearn.base import TransformerMixin
from sklearn.preprocessing import LabelEncoder
import warnings
import traceback
# from django.db import connections
# from edge.stats.db_update.ignore_check import get_unignored
from datetime import datetime
import sys
import openpyxl

In [44]:
warnings.filterwarnings("ignore")
def feature_engineering_test (folder_path, csv_file_names) :
    
    extension = ".csv"
    data_summary = pd.DataFrame()
    
    df_empty = pd.DataFrame()
    with pd.ExcelWriter('FeatureEngineering_ranking_stats.xlsx') as writer:
        df_empty.to_excel(writer, sheet_name='10000KB_to_50000KB', index=False)
    
    with pd.ExcelWriter('FeatureEngineering_output_stats.xlsx') as writer:
        df_empty.to_excel(writer, sheet_name='10000KB_to_50000KB', index=False)

    for i in csv_file_names:
        item = i
        # 1 - Name of the dataset
        file_name = item[0]
        print('Start('+str(datetime.now())+'): '+file_name)
        df = pd.read_csv(folder_path+file_name+extension, encoding='latin-1')
        df_size_bytes = sys.getsizeof(df)
        # 2 - Size(in MB) of the dataset
        # df_size_mb = round(0.00000095367432 * df_size_bytes, 2)
        df_size_mb = 0.00000095367432 * df_size_bytes
        df_shape = df.shape
        # 3 - Count of rows in the dataset
        df_rows = df_shape[0]
        # 4 - Count of columns in the dataset
        df_columns = df_shape[1]
        # 5 - Count of Numerical columns
        df_dtypes = df.dtypes.value_counts()
        num_dtypes = ['int64','float64','int32','float32']
        num_cols = 0
        for i in num_dtypes:
            if df_dtypes.get(i) != None:
                num_cols = num_cols + df_dtypes.get(i)
        # 6 - Count of Categorical columns
        cat_cols = df_columns - num_cols
        # 7 - Count of Missing values
        null_count = df.isnull().sum().sum()
        # 8 - Target column
        target_col = item[1]
        # 9 - Target column datatype
        target_dtype = df[target_col].dtype
        # 10 - Count of distinct classes
        target_classes = df[target_col].nunique()

        def boruta(data, target):
            target_type = data[target].dtype
            target_nclasses = data[target].nunique()
            class DataFrameImputer(TransformerMixin):
                def __init__(self):
                    """Impute missing values.

                    Columns of dtype object are imputed with the most frequent value
                    in column.

                    Columns of other types are imputed with mean of column.

                    """

                def fit(self, X, y=None):
                    self.fill = pd.Series([X[c].value_counts().index[0]
                                           if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
                                          index=X.columns)

                    return self

                def transform(self, X, y=None):
                    return X.fillna(self.fill)

            cat_data = data.select_dtypes(include=["object"])
            num_data = data.select_dtypes(include=["int64", "float64","int32","float32"])
            num_data = DataFrameImputer().fit_transform(num_data)
            cat_data = DataFrameImputer().fit_transform(cat_data)

            # Label encoding the values - Converting into Numerical
            """ Some of the algorithms has hard constraint that it cannot process categorical data . For such algorithms,
                User has to convert categorical features into Numeric. We have few techniques for this operation and 
                we are implementing the famous and good-result methods here as  
                Label - Encoding 
            """
            # cat_data = cat_data.apply(LabelEncoder().fit_transform)
            cat_data = cat_data.apply(lambda col: LabelEncoder().fit_transform(col.astype(str)))
            if cat_data.empty==True:
                data=num_data
            else:
                 data = pd.concat([num_data, cat_data], axis=1)
            Y = data[target]
            X = data.drop([target], axis=1)
            x = X.as_matrix()
            y = Y.as_matrix()
            # x = X.values()
            # y = Y.values()

            # Create empty Multi-Index dataframe
            tuples = []
            index = pd.MultiIndex.from_tuples(tuples, names=['Model', 'Feature_rank'])
            df_result = pd.DataFrame(columns=index)
            model = None

            if (target_type == 'object') and (target_nclasses <= 2):
                model = 'Binary Classification'
                import numpy.ma as ma
                x = np.where(np.isnan(x), ma.array(x, mask=np.isnan(x)).mean(axis=0), x)
                y = np.where(np.isnan(y), ma.array(y, mask=np.isnan(y)).mean(axis=0), y)


                # 1 - Random Forest Classifier
                start_rf = datetime.now()
                rf_ = RandomForestClassifier(n_estimators=500, class_weight='balanced', max_depth=15)
                feature_selection_rf = BorutaPy(rf_, n_estimators='auto', verbose=0)
                feature_selection_rf.fit(x, y)
                stop_rf = datetime.now()
                execution_time_rf = stop_rf-start_rf
                feature_df_rf = pd.DataFrame(X.columns.tolist())
                feature_df_rf['rank'] = feature_selection_rf.ranking_
                feature_df_rf = feature_df_rf.sort_values('rank', ascending=True).reset_index(drop=True)
                feature_df_rf.columns = ['feature_name', 'Rank']
                model_rf = 'Random Forest (Execution Time:'+str(execution_time_rf)+')'
                df_result[model_rf, 'Feature'] = feature_df_rf.feature_name.tolist()
                df_result[model_rf, 'Rank'] = feature_df_rf.Rank.tolist()

                # 2 - XGB Classifier
                start_xgb = datetime.now()
                params_xgb = {'objective':'binary:logistic', 'num_boost_round':50, 'max_depth':10, 'learning_rate':0.1}
                xgb_ = XGBClassifier(**params_xgb)
                feature_selection_xgb = BorutaPy(xgb_, n_estimators='auto', verbose=0)
                feature_selection_xgb.fit(x, y)
                stop_xgb = datetime.now()
                execution_time_xgb = stop_xgb-start_xgb
                feature_df_xgb = pd.DataFrame(X.columns.tolist())
                feature_df_xgb['rank'] = feature_selection_xgb.ranking_
                feature_df_xgb = feature_df_xgb.sort_values('rank', ascending=True).reset_index(drop=True)
                feature_df_xgb.columns = ['feature_name', 'Rank']
                model_xgb = 'XGBoost (Execution Time:'+str(execution_time_xgb)+')'
                df_result[model_xgb, 'Feature'] = feature_df_xgb.feature_name.tolist()
                df_result[model_xgb, 'Rank'] = feature_df_xgb.Rank.tolist()

                # 3 - LGBM Classifier
                start_lgb = datetime.now()
                params_lgb = {'objective':'binary', 'num_boost_round':50, 'max_depth':10, 'learning_rate':0.1, 
                              'class_weight':'balanced'}
                lgb_ = LGBMClassifier(**params_lgb)
                feature_selection_lgb = BorutaPy(lgb_, n_estimators='auto', verbose=0)
                feature_selection_lgb.fit(x, y)
                stop_lgb = datetime.now()
                execution_time_lgb = stop_lgb-start_lgb
                feature_df_lgb = pd.DataFrame(X.columns.tolist())
                feature_df_lgb['rank'] = feature_selection_lgb.ranking_
                feature_df_lgb = feature_df_lgb.sort_values('rank', ascending=True).reset_index(drop=True)
                feature_df_lgb.columns = ['feature_name', 'Rank']
                model_lgb = 'LGBoost (Execution Time:'+str(execution_time_lgb)+')'
                df_result[model_lgb, 'Feature'] = feature_df_lgb.feature_name.tolist()
                df_result[model_lgb, 'Rank'] = feature_df_lgb.Rank.tolist()
            
            elif (target_type == 'object') and (target_nclasses > 2):
                model = 'Multiclass Classification'
                import numpy.ma as ma
                x = np.where(np.isnan(x), ma.array(x, mask=np.isnan(x)).mean(axis=0), x)
                y = np.where(np.isnan(y), ma.array(y, mask=np.isnan(y)).mean(axis=0), y)


                # 1 - Random Forest Classifier
                start_rf = datetime.now()
                rf_ = RandomForestClassifier(n_estimators=500, class_weight='balanced', max_depth=15)
                feature_selection_rf = BorutaPy(rf_, n_estimators='auto', verbose=0)
                feature_selection_rf.fit(x, y)
                stop_rf = datetime.now()
                execution_time_rf = stop_rf-start_rf
                feature_df_rf = pd.DataFrame(X.columns.tolist())
                feature_df_rf['rank'] = feature_selection_rf.ranking_
                feature_df_rf = feature_df_rf.sort_values('rank', ascending=True).reset_index(drop=True)
                feature_df_rf.columns = ['feature_name', 'Rank']
                model_rf = 'Random Forest (Execution Time:'+str(execution_time_rf)+')'
                df_result[model_rf, 'Feature'] = feature_df_rf.feature_name.tolist()
                df_result[model_rf, 'Rank'] = feature_df_rf.Rank.tolist()

                # 2 - XGB Classifier
                start_xgb = datetime.now()
                params_xgb = {'objective':'multi:softmax', 'num_boost_round':50, 'max_depth':10, 'learning_rate':0.1}
                xgb_ = XGBClassifier(**params_xgb)
                feature_selection_xgb = BorutaPy(xgb_, n_estimators='auto', verbose=0)
                feature_selection_xgb.fit(x, y)
                stop_xgb = datetime.now()
                execution_time_xgb = stop_xgb-start_xgb
                feature_df_xgb = pd.DataFrame(X.columns.tolist())
                feature_df_xgb['rank'] = feature_selection_xgb.ranking_
                feature_df_xgb = feature_df_xgb.sort_values('rank', ascending=True).reset_index(drop=True)
                feature_df_xgb.columns = ['feature_name', 'Rank']
                model_xgb = 'XGBoost (Execution Time:'+str(execution_time_xgb)+')'
                df_result[model_xgb, 'Feature'] = feature_df_xgb.feature_name.tolist()
                df_result[model_xgb, 'Rank'] = feature_df_xgb.Rank.tolist()

                # 3 - LGBM Classifier
                start_lgb = datetime.now()
                params_lgb = {'objective':'multiclass', 'num_boost_round':50, 'max_depth':10, 'learning_rate':0.1, 
                              'class_weight':'balanced'}
                lgb_ = LGBMClassifier(**params_lgb)
                feature_selection_lgb = BorutaPy(lgb_, n_estimators='auto', verbose=0)
                feature_selection_lgb.fit(x, y)
                stop_lgb = datetime.now()
                execution_time_lgb = stop_lgb-start_lgb
                feature_df_lgb = pd.DataFrame(X.columns.tolist())
                feature_df_lgb['rank'] = feature_selection_lgb.ranking_
                feature_df_lgb = feature_df_lgb.sort_values('rank', ascending=True).reset_index(drop=True)
                feature_df_lgb.columns = ['feature_name', 'Rank']
                model_lgb = 'LGBoost (Execution Time:'+str(execution_time_lgb)+')'
                df_result[model_lgb, 'Feature'] = feature_df_lgb.feature_name.tolist()
                df_result[model_lgb, 'Rank'] = feature_df_lgb.Rank.tolist()

            elif (target_type == 'int32') | (target_type == 'float32') | (target_type == 'float64') | (target_type == 'int64'):
                model = 'Regression'
                col_mean = np.nanmean(x, axis=0)
                inds = np.where(np.isnan(x))
                x[inds] = np.take(col_mean, inds[1])
                inds = np.where(np.isinf(x))
                x[inds] = np.take(col_mean, inds[1])
                import numpy.ma as ma
                x = np.where(np.isnan(x), ma.array(x, mask=np.isnan(x)).mean(axis=0), x)
                y = np.where(np.isnan(y), ma.array(y, mask=np.isnan(y)).mean(axis=0), y)

                # 1 - Random Forest Regressor
                start_rf = datetime.now()
                rf_ = RandomForestRegressor(n_estimators=500, max_depth=15)
                feature_selection = BorutaPy(rf_, n_estimators='auto', verbose=0)
                feature_selection.fit(x, y)
                stop_rf = datetime.now()
                execution_time_rf = stop_rf-start_rf
                feature_df = pd.DataFrame(X.columns.tolist())
                feature_df['rank'] = feature_selection.ranking_
                feature_df = feature_df.sort_values('rank', ascending=True).reset_index(drop=True)
                feature_df.columns = ['feature_name', 'Rank']
                model_rf = 'Random Forest (Execution Time:'+str(execution_time_rf)+')'
                df_result[model_rf, 'Feature'] = feature_df.feature_name.tolist()
                df_result[model_rf, 'Rank'] = feature_df.Rank.tolist()

                # 2 - XGB Regressor
                start_xgb = datetime.now()
                params_xgb = {'objective':'reg:squarederror', 'n_estimators':50, 'max_depth':10, 'learning_rate':0.1}
                xgb_ = XGBRegressor(**params_xgb)
                feature_selection_xgb = BorutaPy(xgb_, n_estimators='auto', verbose=0)
                feature_selection_xgb.fit(x, y)
                stop_xgb = datetime.now()
                execution_time_xgb = stop_xgb-start_xgb
                feature_df_xgb = pd.DataFrame(X.columns.tolist())
                feature_df_xgb['rank'] = feature_selection_xgb.ranking_
                feature_df_xgb = feature_df_xgb.sort_values('rank', ascending=True).reset_index(drop=True)
                feature_df_xgb.columns = ['feature_name', 'Rank']
                model_xgb = 'XGBoost (Execution Time:'+str(execution_time_xgb)+')'
                df_result[model_xgb, 'Feature'] = feature_df_xgb.feature_name.tolist()
                df_result[model_xgb, 'Rank'] = feature_df_xgb.Rank.tolist()

                # 3 - LGBM Regressor
                start_lgb = datetime.now()
                params_lgb = {'objective':'regression', 'n_estimators':50, 'max_depth':10, 'learning_rate':0.1, 
                              'class_weight':'balanced'}
                lgb_ = LGBMRegressor(**params_lgb)
                feature_selection_lgb = BorutaPy(lgb_, n_estimators='auto', verbose=0)
                feature_selection_lgb.fit(x, y)
                stop_lgb = datetime.now()
                execution_time_lgb = stop_lgb-start_lgb
                feature_df_lgb = pd.DataFrame(X.columns.tolist())
                feature_df_lgb['rank'] = feature_selection_lgb.ranking_
                feature_df_lgb = feature_df_lgb.sort_values('rank', ascending=True).reset_index(drop=True)
                feature_df_lgb.columns = ['feature_name', 'Rank']
                model_lgb = 'LGBoost (Execution Time:'+str(execution_time_lgb)+')'
                df_result[model_lgb, 'Feature'] = feature_df_lgb.feature_name.tolist()
                df_result[model_lgb, 'Rank'] = feature_df_lgb.Rank.tolist()

            # return feature_df.head(feature_selection.n_features_),feature_df,execution_time_rf
            # return df_result
            return model, execution_time_rf, execution_time_xgb, execution_time_lgb, df_result

        model, execution_time_rf, execution_time_xgb, execution_time_lgb, df_result = boruta(data=df, target=target_col)

        row_data = [[file_name, df_size_mb, df_rows, df_columns, cat_cols, num_cols, null_count, target_col, target_dtype, 
                    target_classes, model, execution_time_rf, execution_time_xgb, execution_time_lgb, 
                    str(execution_time_rf), str(execution_time_xgb), str(execution_time_lgb)], ]
        df_2 = pd.DataFrame(row_data, columns=['Dataset','Size(MB)','#_Rows','#_Columns','#_Category_cols','#_Numerical_cols',
                                               '#_Missing_values','Target_col','Target_dtype','#_Target_Unique_values',
                                               'Model_type','ExecTime_RandomForest','ExecTime_XGBoost','ExecTime_LightGBM',
                                               'str_ExecTime_RF','str_ExecTime_XGB','str_ExecTime_LGB'])
        df_2['str_ET_RF'] = df_2['ExecTime_RandomForest'].apply(lambda x: str(x) if x!=None else x)
        df_2['str_ET_XGB'] = df_2['ExecTime_XGBoost'].apply(lambda x: str(x) if x!=None else x)
        df_2['str_ET_LGB'] = df_2['ExecTime_LightGBM'].apply(lambda x: str(x) if x!=None else x)
        
        data_summary = data_summary.append(df_2, ignore_index=True)
        
        with pd.ExcelWriter('FeatureEngineering_ranking_stats.xlsx', engine='openpyxl', mode='a') as writer:
            df_result.to_excel(writer, sheet_name=file_name, index=True)
        
        with pd.ExcelWriter('FeatureEngineering_output_stats.xlsx', engine='openpyxl', mode='a') as writer:
            data_summary.to_excel(writer, sheet_name=file_name, index=False)
        print('End('+str(datetime.now())+'): '+file_name)
    
    return data_summary

In [35]:
folder_path = "D:/Solverminds/Datasets/"
csv_file_names = [('Titanic','Survived'),('Loan','Loan_Status')]

In [36]:
df_stats = feature_engineering_test(folder_path=folder_path, csv_file_names=csv_file_names)
df_stats.head()

Start(2020-06-28 16:16:20.168519): Titanic
End(2020-06-28 16:16:42.798640): Titanic
Start(2020-06-28 16:16:42.798640): Loan
End(2020-06-28 16:16:52.189331): Loan


,Dataset,Size(MB),#_Rows,#_Columns,#_Category_cols,#_Numerical_cols,#_Missing_values,Target_col,Target_dtype,#_Target_Unique_values,Model_type,ExecTime_RandomForest,ExecTime_XGBoost,ExecTime_LightGBM,str_ExecTime_RF,str_ExecTime_XGB,str_ExecTime_LGB,str_ET_RF,str_ET_XGB,str_ET_LGB
0,Titanic,0.311073,891,12,7,5,866,Survived,int64,2,Regression,00:00:02.608227,00:00:13.616966,00:00:06.124956,0:00:02.608227,0:00:13.616966,0:00:06.124956,0 days 00:00:02.608227,0 days 00:00:13.616966,0 days 00:00:06.124956
1,Loan,0.313653,614,13,12,1,149,Loan_Status,object,2,Binary Classification,00:00:03.890419,00:00:04.048261,00:00:01.060014,0:00:03.890419,0:00:04.048261,0:00:01.060014,0 days 00:00:03.890419,0 days 00:00:04.048261,0 days 00:00:01.060014


In [20]:
folder_path = "D:/Solverminds/Datasets/"
csv_below_1000KB = [('NAN_file','diagnosis'),('recruiting_costs','Total'),('recruiting_costs','Employment Source'),
                    ('fertility_Diagnosis','target'),('headbrain','Brain Weight(grams)'),('Adult','income'),('Iris','Species'),
                    ('Blood_transfusion_Trn','Blood'),('Heart_disease_Prediction','heart_disease'),('wine_1','Proline'),
                    ('Diabetes','Outcome'),('armenianpubs','Fav_Pub'),('production_staff','90-day Complaints'),
                    ('ii_snapshot','resigned_rejoined'),('Loan','Loan_Status'),('CaystonFinalData','class'),
                    ('patientdetail','Fatigue'),('Concrete_Data','Concrete compressive strength(MPa, megapascals) '),
                    ('review','Score'),('Titanic','Survived'),('StorageAnalysistest','VslDelayBy'),('earthquake','depth'),
                    ('core_dataset','Pay Rate'),('food_coded','weight'),('HR_Full','Pay Rate'),('patient','Status'),
                    ('Rock_Mine','Rock_Mine'),('winequality-red','quality'),('Sacramento_realestate_transactions','price'),
                    ('Cancer_Diagnostic','diagnosis'),('Detect_anomaly','count'),('Curr_test','sensex'),('soybean','class'),
                    ('abalone_data','rings'),('Azdata','FamilyHistory'),('Missing_People','Gender'),('test_detect_anoms','count'),
                    ('all','Bus.services'),('all','Drinking.water.facilities'),('mushrooms','class'),('PMS_TS','TITLE'),
                    ('house','SalePrice'),('all_data_ii','resigned_rejoined'),('HR','salary'),('Mart_Sales','Item_Outlet_Sales'),
                    ('CC_GENERAL','CREDIT_LIMIT')]

In [21]:
df_stats_below_1000KB = feature_engineering_test(folder_path=folder_path, csv_file_names=csv_below_1000KB)
df_stats_below_1000KB['str_ET_RF'] = df_stats_below_1000KB['ExecTime_RandomForest'].apply(lambda x: str(x) if x!=None else x)
df_stats_below_1000KB['str_ET_XGB'] = df_stats_below_1000KB['ExecTime_XGBoost'].apply(lambda x: str(x) if x!=None else x)
df_stats_below_1000KB['str_ET_LGB'] = df_stats_below_1000KB['ExecTime_LightGBM'].apply(lambda x: str(x) if x!=None else x)
df_stats_below_1000KB.head()

Start(2020-06-28 10:55:46.379839): NAN_file
End(2020-06-28 10:55:48.120704): NAN_file
Start(2020-06-28 10:55:48.120704): recruiting_costs
End(2020-06-28 10:55:52.781284): recruiting_costs
Start(2020-06-28 10:55:52.781284): recruiting_costs
End(2020-06-28 10:55:54.342298): recruiting_costs
Start(2020-06-28 10:55:54.342298): fertility_Diagnosis
End(2020-06-28 10:55:58.795311): fertility_Diagnosis
Start(2020-06-28 10:55:58.795311): headbrain
End(2020-06-28 10:55:59.635283): headbrain
Start(2020-06-28 10:55:59.635283): Adult
End(2020-06-28 10:56:01.756391): Adult
Start(2020-06-28 10:56:01.756391): Iris
End(2020-06-28 10:56:04.652667): Iris
Start(2020-06-28 10:56:04.660632): Blood_transfusion_Trn
End(2020-06-28 10:56:10.287158): Blood_transfusion_Trn
Start(2020-06-28 10:56:10.287158): Heart_disease_Prediction
End(2020-06-28 10:56:20.904482): Heart_disease_Prediction
Start(2020-06-28 10:56:20.904482): wine_1
End(2020-06-28 10:56:35.548043): wine_1
Start(2020-06-28 10:56:35.548043): Diabetes


,Dataset,Size(MB),#_Rows,#_Columns,#_Category_cols,#_Numerical_cols,#_Missing_values,Target_col,Target_dtype,#_Target_Unique_values,Model_type,ExecTime_RandomForest,ExecTime_XGBoost,ExecTime_LightGBM,str_ExecTime_RF,str_ExecTime_XGB,str_ExecTime_LGB,str_ET_RF,str_ET_XGB,str_ET_LGB
0,NAN_file,0.006883,17,11,10,1,0,diagnosis,object,2,Binary Classification,00:00:00.948802,00:00:00.439981,00:00:00.079974,0:00:00.948802,0:00:00.439981,0:00:00.079974,0 days 00:00:00.948802,0 days 00:00:00.439981,0 days 00:00:00.079974
1,recruiting_costs,0.003888,22,14,14,0,0,Total,int64,15,Regression,00:00:03.798507,00:00:00.518079,00:00:00.071993,0:00:03.798507,0:00:00.518079,0:00:00.071993,0 days 00:00:03.798507,0 days 00:00:00.518079,0 days 00:00:00.071993
2,recruiting_costs,0.003888,22,14,14,0,0,Employment Source,object,22,Multiclass Classification,00:00:00.423982,00:00:00.809958,00:00:00.132092,0:00:00.423982,0:00:00.809958,0:00:00.132092,0 days 00:00:00.423982,0 days 00:00:00.809958,0 days 00:00:00.132092
3,fertility_Diagnosis,0.012924,100,10,5,5,0,target,object,2,Binary Classification,00:00:02.168140,00:00:00.463959,00:00:01.628931,0:00:02.168140,0:00:00.463959,0:00:01.628931,0 days 00:00:02.168140,0 days 00:00:00.463959,0 days 00:00:01.628931
4,headbrain,0.007378,237,4,4,0,0,Brain Weight(grams),int64,146,Regression,00:00:00.329002,00:00:00.087993,00:00:00.127968,0:00:00.329002,0:00:00.087993,0:00:00.127968,0 days 00:00:00.329002,0 days 00:00:00.087993,0 days 00:00:00.127968


In [23]:
with pd.ExcelWriter('FeatureEngineering_output_stats.xlsx', engine='openpyxl', mode='a') as writer:
    df_stats_below_1000KB.to_excel(writer, sheet_name='(v2)below_1000KB', index=False)

In [42]:
folder_path = "D:/Solverminds/Datasets/"
csv_1000KB_to_10000KB_1 = [('Nursery_Trn','Class'),('traintop20022','target'),('injury_incident_nearmiss','vessel'),
                         ('video_games_sales','Rating'),('Bay_Data','bay1'),('OWT','c1'),('netflix_titles_nov_2019','rating'),
                         ('bank-full','y'),('Adult_data_Set','income'),('Adult_data_Set','maritalstatus'),('Banking','y'),
                         ('tmdb_5000_movies','vote_average'),('consolidated_coin_data','Market Cap'),
                         ('Sepsis','Death')]


csv_1000KB_to_10000KB_2 = [('consolidated_coin_data','Market Cap'),('Sepsis','Death')]

csv_1000KB_to_10000KB_3 = [('Sepsis','Death')]

In [43]:
df_stats_1000KB_to_10000KB = feature_engineering_test(folder_path=folder_path, csv_file_names=csv_1000KB_to_10000KB_3)
df_stats_1000KB_to_10000KB.head()

Start(2020-06-29 07:50:32.705910): Sepsis
End(2020-06-29 08:12:49.397277): Sepsis


,Dataset,Size(MB),#_Rows,#_Columns,#_Category_cols,#_Numerical_cols,#_Missing_values,Target_col,Target_dtype,#_Target_Unique_values,Model_type,ExecTime_RandomForest,ExecTime_XGBoost,ExecTime_LightGBM,str_ExecTime_RF,str_ExecTime_XGB,str_ExecTime_LGB,str_ET_RF,str_ET_XGB,str_ET_LGB
0,Sepsis,47.82975,44677,76,76,0,10162,Death,int64,2,Regression,00:12:08.312042,00:09:36.937196,00:00:21.097224,0:12:08.312042,0:09:36.937196,0:00:21.097224,0 days 00:12:08.312042,0 days 00:09:36.937196,0 days 00:00:21.097224


In [47]:
folder_path = "D:/Solverminds/Datasets/"
csv_10000KB_to_50000KB_1 = [('nyc-rolling-sales-2','SALE PRICE'),('fifa_data','special'),('insurance_data','Response'),
                          ('HES_DATA_1','classpat'),('Date_data_check','weight'),('athlete_events','Medal')]
csv_10000KB_to_50000KB_2 = [('fifa_data','special'),('insurance_data','Response'),
                          ('HES_DATA_1','classpat'),('Date_data_check','weight'),('athlete_events','Medal')]

In [48]:
df_stats_10000KB_to_50000KB = feature_engineering_test(folder_path=folder_path, csv_file_names=csv_10000KB_to_50000KB_2)
df_stats_10000KB_to_50000KB.head()

Start(2020-06-29 11:53:12.013083): fifa_data
End(2020-06-29 15:02:34.305867): fifa_data
Start(2020-06-29 15:02:34.315594): insurance_data
End(2020-06-29 16:55:43.281106): insurance_data
Start(2020-06-29 16:55:43.418622): HES_DATA_1
End(2020-06-29 19:16:52.031412): HES_DATA_1
Start(2020-06-29 19:16:52.033411): Date_data_check
End(2020-06-29 19:40:30.070561): Date_data_check
Start(2020-06-29 19:40:30.070561): athlete_events
End(2020-06-29 21:14:42.754998): athlete_events


,Dataset,Size(MB),#_Rows,#_Columns,#_Category_cols,#_Numerical_cols,#_Missing_values,Target_col,Target_dtype,#_Target_Unique_values,Model_type,ExecTime_RandomForest,ExecTime_XGBoost,ExecTime_LightGBM,str_ExecTime_RF,str_ExecTime_XGB,str_ExecTime_LGB,str_ET_RF,str_ET_XGB,str_ET_LGB
0,fifa_data,29.305033,17994,185,185,0,70772,special,int64,1394,Regression,02:28:04.575302,00:37:03.703098,00:04:04.456138,2:28:04.575302,0:37:03.703098,0:04:04.456138,0 days 02:28:04.575302,0 days 00:37:03.703098,0 days 00:04:04.456138
1,insurance_data,60.877540,59381,128,19,109,393103,Response,int64,8,Regression,00:45:43.451490,01:05:47.604491,00:01:30.691619,0:45:43.451490,1:05:47.604491,0:01:30.691619,0 days 00:45:43.451490,0 days 01:05:47.604491,0 days 00:01:30.691619
2,HES_DATA_1,324.552448,324235,24,23,1,2095945,classpat,float64,4,Regression,00:21:57.640027,01:53:10.733750,00:05:46.233401,0:21:57.640027,1:53:10.733750,0:05:46.233401,0 days 00:21:57.640027,0 days 01:53:10.733750,0 days 00:05:46.233401
3,Date_data_check,182.748485,394531,11,8,3,7,weight,float64,70802,Regression,00:14:04.623742,00:06:49.061185,00:02:39.390138,0:14:04.623742,0:06:49.061185,0:02:39.390138,0 days 00:14:04.623742,0 days 00:06:49.061185,0 days 00:02:39.390138
4,athlete_events,178.806782,271116,15,13,2,363853,Medal,object,3,Multiclass Classification,00:07:36.529535,01:25:05.985313,00:01:24.655466,0:07:36.529535,1:25:05.985313,0:01:24.655466,0 days 00:07:36.529535,0 days 01:25:05.985313,0 days 00:01:24.655466


In [49]:
folder_path = "D:/Solverminds/Datasets/"
csv_50000KB_to_150000KB = [('Electricity_P','UNE'),('resources','price'),('creditcard','Amount')]

In [ ]:
df_stats_50000KB_to_150000KB = feature_engineering_test(folder_path=folder_path, csv_file_names=csv_50000KB_to_150000KB)
df_stats_50000KB_to_150000KB.head()

Start(2020-06-29 21:49:41.115116): Electricity_P
